In [1]:
import simtk.openmm as openmm

In [2]:
from simtk.openmm import app

In [3]:
from simtk import unit

In [4]:
openmm.__version__

'7.0'

In [5]:
pwd

'/Users/rafalpwiewiora/repos/erk_reconstruction/PDB_files_for_amber_parametrization/repack/erk2_q120m'

In [6]:
ls

equilibrated.pdb             reserialize_openmm631.ipynb
ff99SBildn.xml               reserialized/
integrator.xml               state.xml
make_701_system.ipynb        system.xml
phosaa10.xml                 tip3p_standard.xml


In [7]:
system_xml_filename = 'system.xml'
integrator_xml_filename = 'integrator.xml'
state_xml_filename = 'state.xml'

pdb_filename = 'equilibrated.pdb'

water_model = 'tip3p'
solvent_padding = 10.0 * unit.angstroms
solvent_numAdded = 20959

ffxml_filenames = ['ff99SBildn.xml', 'tip3p_standard.xml', 'phosaa10.xml']

pressure = 1.0 * unit.atmospheres
barostat_period = 50
temperature = 300 * unit.kelvin
equilibration_collision_rate = 20.0 / unit.picoseconds
production_collision_rate = 1.0 / unit.picoseconds
timestep = 2.0 * unit.femtoseconds
nsteps = 500 # 1 ps
niterations = 10000 # 10 ns

#minimization_tolerance = 10.0 * unit.kilojoules_per_mole / unit.nanometers
#minimization_steps = 20
nonbondedMethod = app.PME
cutoff = 0.9 * unit.nanometers
constraints = app.HBonds
rigidWater = True
removeCMMotion = False

In [8]:
pdb = app.PDBFile(pdb_filename)
forcefield = app.ForceField(*ffxml_filenames)

In [9]:
modeller = app.Modeller(pdb.topology, pdb.positions)

In [10]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=nonbondedMethod, cutoff=cutoff, constraints=constraints, rigidWater=rigidWater, removeCMMotion=removeCMMotion)

# Add a barostat
print('Adding barostat...')
barostat = openmm.MonteCarloBarostat(pressure, temperature, barostat_period)
system.addForce(barostat)

Adding barostat...


4

In [11]:
state = openmm.XmlSerializer.deserialize(open(state_xml_filename).read())

In [12]:
vectors = state.getPeriodicBoxVectors()

In [13]:
system.setDefaultPeriodicBoxVectors(*vectors)

In [14]:
with open('reserialized/system.xml', 'w') as f:
    xml = openmm.XmlSerializer.serialize(system)
    f.write(xml)